In [1]:
using PorousMaterials # Pkg.clone("https://github.com/SimonEnsemble/PorousMaterials.jl", "v0.1.1")
using DelimitedFiles
using LinearAlgebra
using JLD2
using CoherentPointDrift
import Bio3DView
using Printf
import PyPlot
using Random

# Database of cages

Cage database CDB41 (41 cages) from [here](https://github.com/marcinmiklitz/CDB41). Cleaned of solvent. Thanks to Kim Jelfs and M. Miklitz for kindly providing the cages.

Reference:
> M. Miklitz, S. Jiang, R. Clowes, M. E. Briggs, A. I. Cooper and K. E. Jelfs, Computational Screening of Porous Organic Molecules for Xenon/Krypton Separation, J. Phys. Chem. C, 2017. DOI: 10.1021/acs.jpcc.7b03848

Other set of 33 cages from the .xyz files deposited as ESI of DOI 10.1038/s41467-018-05271-9

> R. L. Greenaway, V. Santolini, M. J. Bennison, B. M. Alston, C. J. Pugh, M. A. Little, M. Miklitz, E. G. B. Eden-Rump, R. Clowes, A. Shakil, H. J. Cuthbertson, H. Armstrong, M. E. Briggs, K. E. Jelfs & A. I. Cooper. High-throughput discovery of organic cages and catenanes using computational screening fused with robotic synthesis. Nature Communications, 2018. DOI: 10.1038/s41467-018-05271-9
                                                              
I manually looked at Fig 4 in the main text and selected those that were synthesized. Some cages required visualization to compare to Fig 4 since e.g. there are multiple versions of B23, a few of which were apparently not synthesized but only computational predictions.

The `.xyz` files describing the molecular structure of all of these cages are in the directory `all_cages`.

In [2]:
cages = readdlm("all_cages/all_cages.txt", String)[:];

# Center cages

The center of mass will be set to be the origin.
Each cage will be rotated such that it is aligned with its principle moments of inertia.
i.e. the moment of inertia matrix of an "aligned" cage is diagonal.


First, a function to compute the center of mass of a group of `atoms` at Cartesian coordinates `x`, stored in the columns.

In [3]:
function center_of_mass(atoms::Array{Symbol, 1}, x::Array{Float64, 2})
    ams = read_atomic_masses()
    com = [0.0, 0.0, 0.0]
    total_mass = 0.0
    for (i, atom) in enumerate(atoms)
        com += ams[atom] * x[:, i]
        total_mass += ams[atom]
    end
    return com / total_mass
end

center_of_mass (generic function with 1 method)

In [4]:
function centered_cage_coords(cage::AbstractString)
    # read in raw .xyz from `all_cages`
    atoms, x = read_xyz("all_cages/" * cage * ".xyz")
    
    # compute center of mass
    x_com = center_of_mass(atoms, x)
    
    # shift coords so that cage is centered
    x = x .- x_com
    
    # write centered cage coords
    write_xyz(atoms, x, "centered_cages/" * cage * ".xyz")
    
    return atoms, x
end

centered_cage_coords (generic function with 1 method)

# Porosity point clouds

In [5]:
function generate_porosity_point_cloud(atoms::Array{Symbol, 1}, 
                                       x::Array{Float64, 2}, 
                                       nb_pts_in_porosity_cloud::Int, 
                                       snapshot_radius::Union{Nothing, Float64};
                                       gaussian_scaler::Float64=3.0)
    # store points in void space here.
    x_porosity_cloud = zeros(3, nb_pts_in_porosity_cloud)

    # determine if void space via potential energy of He
    he = Molecule("He")
    ljff = LJForceField("UFF.csv", cutoffradius=14.0, mixing_rules="geometric")
    ljspheres = Atoms(atoms, x)

    # compute radius of the molecule; this determines variance of Gaussian if snapshot size not provided
    if snapshot_radius == nothing
        snapshot_radius = maximum([norm(x[:, a]) for a = 1:length(atoms)])
    end
    
    n_hits = 0
    while n_hits < nb_pts_in_porosity_cloud
        # bias insertions towards center
        x_insert = randn(3) * snapshot_radius / gaussian_scaler # scale smaller to bias describing core more.
        if norm(x_insert) > snapshot_radius
            continue
        end

        # put helium at this grid pt
        translate_to!(he, x_insert)

        # compute potential energy of He adsorbate here
        energy = vdw_energy_no_PBC(he, ljspheres, ljff)

        if energy < 298.0
            n_hits += 1
            x_porosity_cloud[:, n_hits] = x_insert
        end
    end
    
    return x_porosity_cloud
end

# atoms, x = centered_cage_coords("B1")
# x = generate_porosity_point_cloud(atoms, x, 10000, nothing)
# write_xyz([:H for i = 1:10000], x, "test.xyz")

generate_porosity_point_cloud (generic function with 1 method)

# Rotate to align principal axes of rotation with Cartesian axes

Next, a function to compute the moment of inertia matrix of a group of `atoms` at Cartesian positions `x`. 

See <a href="https://chem.libretexts.org/Textbook_Maps/Physical_and_Theoretical_Chemistry_Textbook_Maps/Map%3A_Physical_Chemistry_(McQuarrie_and_Simon)/13%3A_Molecular_Spectroscopy/13-08._The_Rotational_Spectrum_of_a_Polyatomic_Molecule_Depends_Upon_the_Principal_Moments_of_Inertia_of_the_Molecule">Chemistry Libre Texts</a> moment of inertia formulas.

In [6]:
function moment_of_inertia(x::Array{Float64, 2})
    # moment of inertia matrix `mi`
    mi = zeros(Float64, 3, 3)
    for i = 1:3
        for j = 1:3
            for a = 1:size(x)[2]
                if i == j
                    ids = collect(setdiff(Set([1, 2, 3]), Set([i])))
                    mi[i, j] += sum(x[ids, a] .^ 2)
                else
                    mi[i, j] -= x[i, a] * x[j, a]
                end
            end
        end
    end
    
    # should be symmetric!
    @assert isapprox(mi, mi')
        
    return mi
end

moment_of_inertia (generic function with 1 method)

In [7]:
viewcage(cage::AbstractString) = Bio3DView.viewfile("final_aligned_cages/" * cage * ".xyz", "xyz")

viewcage (generic function with 1 method)

In [36]:
for dir in ["final_aligned_cages", "rotational_inertia_aligned_cages", "centered_cages"]
    isdir(dir) ? nothing : mkdir(dir)
    isdir(joinpath(dir, "porosity_point_clouds")) ? nothing : mkdir(joinpath(dir, "porosity_point_clouds"))
end

write_final_aligned_cage(atoms::Array{Symbol, 1}, x::Array{Float64, 2}, cage::AbstractString) = write_xyz(atoms, x, "final_aligned_cages/" * cage * ".xyz")
read_final_aligned_cage(cage::AbstractString) = read_xyz("final_aligned_cages/" * cage * ".xyz")

write_rotational_inertia_aligned_cage(atoms::Array{Symbol, 1}, x::Array{Float64, 2}, cage::AbstractString) = write_xyz(atoms, x, "rotational_inertia_aligned_cages/" * cage * ".xyz")
read_rotational_inertia_aligned_cage(cage::AbstractString) = read_xyz("rotational_inertia_aligned_cages/" * cage * ".xyz")

read_rotational_inertia_aligned_cage (generic function with 1 method)

In [9]:
Random.seed!(1337); # set random number seed for complete reproducibility

nb_pts_in_pt_cloud = 100000

aligned = Dict(cage => false for cage in cages)

for cage in cages
    # read in centered cage
    atoms, x = centered_cage_coords(cage)
    
    # generate porosity point cloud
    x_pt_cloud = generate_porosity_point_cloud(atoms, x, nb_pts_in_pt_cloud, nothing) # use radius of molecule for this.
    write_xyz([:H for i = 1:nb_pts_in_pt_cloud], x_pt_cloud, 
        "centered_cages/porosity_point_clouds/" * cage * ".xyz")
    
    # diagnomize moment of inertia matrix
    mi = moment_of_inertia(x_pt_cloud)
    λ, v = eigen(mi) # columns of v are eigenvalues
    ids = sortperm(λ, rev=true) # sort eigenvalues from large to small.
    λ = λ[ids]
    v = v[:, ids]
    # tests out of paranoia
    @assert(isapprox(mi * v, v * diagm(0 => λ)), "eigenvectors not right")
    @assert(isapprox([norm(v[:, i]) for i = 1:3], ones(3)), "eigenvectors not unit vectors")
    @assert(isapprox(mi, v * diagm(0 => λ) * v'))
    # `v` is a rotation matrix b/c `mi` is symmetric and the columns are orthonormal
    # i.e. `v` is a unitary matrix. now use it to rotate x to align with principle axes of rotation
    @assert(isapprox(v' * v, Diagonal{Float64}(I, 3)), "v is not unitary...")
    @assert (λ[1] >= λ[2]) & (λ[2] >= λ[3])
    
    # align principal axes of rotation of the cage with the Cartesian axes
    write_rotational_inertia_aligned_cage(atoms, v' * x, cage)
    
    # look for degeneracy
    xy_symmetry = isapprox(λ[1], λ[2], rtol=0.01) # can use rtol b/c fixed # pts now.
    yz_symmetry = isapprox(λ[2], λ[3], rtol=0.01)
    if xy_symmetry || yz_symmetry
        printstyled(cage * " has nearly degenerate axes of inertia\n", color=:red)
    else
        aligned[cage] = true
        write_final_aligned_cage(atoms, v' * x, cage)
        printstyled(cage * " has unique axes of inertia\n", color=:green)
    end
end

A11 has nearly degenerate axes of inertia
B11 has nearly degenerate axes of inertia
B13 has unique axes of inertia
B15 has nearly degenerate axes of inertia
B18 has nearly degenerate axes of inertia
B1 has nearly degenerate axes of inertia
B23 has nearly degenerate axes of inertia
B24 has nearly degenerate axes of inertia
B25 has nearly degenerate axes of inertia
B26 has nearly degenerate axes of inertia
B2 has nearly degenerate axes of inertia
B4 has nearly degenerate axes of inertia
B5 has nearly degenerate axes of inertia
B6 has nearly degenerate axes of inertia
B8 has nearly degenerate axes of inertia
B9 has nearly degenerate axes of inertia
C11 has nearly degenerate axes of inertia
C13 has unique axes of inertia
C15 has nearly degenerate axes of inertia
C18 has nearly degenerate axes of inertia
C1 has nearly degenerate axes of inertia
C20 has unique axes of inertia
C21 has nearly degenerate axes of inertia
C23 has nearly degenerate axes of inertia
C24 has nearly degenerate axes of

Write to file list of cages where principal axes of inertia are authoratative on how to align.

In [10]:
rotational_inertia_aligned_cages = cages[[aligned[cage] for cage in cages]]
@save "rotational_inertia_aligned_cages.jld2" rotational_inertia_aligned_cages

# Rigid point set registration to align cages consistently where moments of inertia about principal axes are nearly degenerate

Write porosity point clouds of rotational-inertia-aligned cages for further alignment where principal axes are nearly dgenerate. Store in `rotational_inertia_aligned_cages/porosity_point_clouds`.

In [11]:
read_porosity_pt_cloud_rot_aligned(cage::AbstractString) = read_xyz("rotational_inertia_aligned_cages/porosity_point_clouds/" * cage * ".xyz")[2]
write_porosity_pt_cloud_rot_aligned(x_pt_cloud::Array{Float64, 2}, cage::AbstractString) = write_xyz(
    [:H for i = 1:nb_pts_in_pt_cloud], 
    x_pt_cloud, 
    "rotational_inertia_aligned_cages/porosity_point_clouds/" * cage * ".xyz")

write_porosity_pt_cloud_rot_aligned (generic function with 1 method)

In [12]:
Random.seed!(1337); # set random number seed for complete reproducibility

nb_pts_in_pt_cloud = 5000

porosity_point_clouds = Dict{AbstractString, Array{Float64, 2}}()

# use same dist'n of points for this one to compare cages easily
for cage in cages
    atoms, x = read_rotational_inertia_aligned_cage(cage)
    
    porosity_point_clouds[cage] = generate_porosity_point_cloud(atoms, x, nb_pts_in_pt_cloud, nothing, gaussian_scaler=5.0)
    
    write_porosity_pt_cloud_rot_aligned(porosity_point_clouds[cage], cage)
end

In [21]:
function read_jld_file_results(cage_y::AbstractString, cage_x::AbstractString; verbose::Bool=false)
    jldfilename = jldopen(@sprintf("cpd_results/align_%s_to_%s_5000_pts.jld2", cage_y, cage_x))
    R = read(jldfilename, "R")
    σ² = read(jldfilename, "σ²")
    ℓ = read(jldfilename, "ℓ")
    @assert read(jldfilename, "cage_y") == cage_y
    @assert read(jldfilename, "cage_x") == cage_x
    close(jldfilename)
    return R, σ², ℓ
end

function find_pair_to_align(cage_ys::Array{String, 1}, cage_xs::Array{String, 1})
    # look at all possible pairs, align cage_y to cage_x, record likelihood of alignment.
    pairz = Tuple{String, String}[]
    ℓs = Float64[]
    σ²s = Float64[]
    
    for cage_y in cage_ys
        for cage_x in cage_xs
            if cage_y == cage_x
                continue
            end
            
            R, σ², ℓ = read_jld_file_results(cage_y, cage_x)
            
            push!(ℓs, ℓ)
            push!(σ²s, σ²)
            push!(pairz, (cage_y, cage_x))
        end
    end
    # align the pair where the negative log likelihood is smallest.
    best_pair = pairz[argmin(ℓs)]
    return best_pair[1], best_pair[2]
end

find_pair_to_align (generic function with 1 method)

recompute point cloud b/c errors accumate after applying tons of rotation matrices

In [ ]:
@load "rotational_inertia_aligned_cages.jld2"
aligned = Dict(cage => (cage in rotational_inertia_aligned_cages) for cage in cages)
# rotation_matrix_applied = Dict(cage => diagm(0 => [1.0, 1.0, 1.0]) for cage in cages) # diagonal matrix applied

nb_pts_in_pt_cloud = 1000

while sum([! aligned[cage] for cage in cages]) > 0
    ###
    #  remake list of aligned and unaligned cages
    ###
    aligned_cages = cages[[aligned[cage] for cage in cages]] # cage_x's since these will not be rotated
    unaligned_cages = cages[[! aligned[cage] for cage in cages]] # cage_y's since all are to be rotated
    
    ###
    #  loop thru all possible (unaligned_cages, aligned_cages) pairs to align
    #     select pair to align via min -ve log likelihood in coherent point drift algo
    ###
    cage_y, cage_x = find_pair_to_align(unaligned_cages, aligned_cages)
    printstyled(@sprintf("Aligning cage %s to %s...\n", cage_y, cage_x), color=:yellow)
    @assert cage_x in aligned_cages
    @assert cage_y in unaligned_cages
    
    # read in cage_x from its final aligned position (this is an aligned cage!)
    atoms_x, x = read_final_aligned_cage(cage_x)
    x_pt_cloud = generate_porosity_point_cloud(atoms_x, x, nb_pts_in_pt_cloud, 
                                               nothing, gaussian_scaler=5.0)
    
    atoms_y, y = read_rotational_inertia_aligned_cage(cage_y)
    
#     # best guess for the rotation matrix, perform rotation now.
#     R, σ², ℓ = read_jld_file_results(cage_y, cage_x) # but cage_X may hv been transformed previously!
#     rotation_matrix_applied[cage_y] = R
#     y = R * rotation_matrix_applied[cage_x] * y # overwrite y with best guess for rotation
    
    y_pt_cloud = generate_porosity_point_cloud(atoms_y, y, nb_pts_in_pt_cloud, 
                                               nothing, gaussian_scaler=5.0)
    
    # transformation is applied to the unaligned cage!
    R, t, σ², ℓ = CoherentPointDrift.rigid_point_set_registration(x_pt_cloud, y_pt_cloud, verbose=false,
                       w=0.0, σ²_tol=0.01, q_tol=0.1, max_nb_em_steps=30, print_ending=true)
    
    println("\tσ² =", σ²)
#     write_final_aligned_cage(atoms_y, R * rotation_matrix_applied[cage_x] * y, cage_y)
    write_final_aligned_cage(atoms_y, R * y, cage_y)
    
    aligned[cage_y] = true
end

Aligning cage CC2 to CP1...
	σ² = 0.015086, q = -3677.423211, EM steps taken = 29, reason for exit: objective stopped decreasing, time: 0.51146 min
	σ² =0.015085777281872348
Aligning cage WC4 to CP3...
	σ² = 0.008693, q = -3589.858398, EM steps taken = 7, reason for exit: variance below tol, time: 0.12245 min
	σ² =0.00869322002778381
Aligning cage HC1 to CP4...
	σ² = 0.012425, q = -4056.918338, EM steps taken = 30, reason for exit: max EM steps reached, time: 0.51935 min
	σ² =0.012425356489769834
Aligning cage CD2 to CP3...
	σ² = 0.009909, q = -3357.107481, EM steps taken = 11, reason for exit: variance below tol, time: 0.22643 min
	σ² =0.009909385088460265
Aligning cage CC9 to CP1...
	σ² = 0.018169, q = -3361.442735, EM steps taken = 30, reason for exit: max EM steps reached, time: 0.50326 min
	σ² =0.01816935658507176
Aligning cage CC4 to CP3...
	σ² = 0.009938, q = -3102.480090, EM steps taken = 30, reason for exit: variance below tol, time: 0.51164 min
	σ² =0.009938042490319883
Align

some quick testing

In [18]:
cage_x = "B1"
cage_y = "B2"

atoms_x, x = read_rotational_inertia_final_aligned_cage(cage_x)
x_pt_cld = read_porosity_pt_cloud_rot_aligned(cage_x)
atoms_y, y = read_rotational_inertia_final_aligned_cage(cage_y)
y_pt_cld = read_porosity_pt_cloud_rot_aligned(cage_y)

R, t, σ², ℓ = CoherentPointDrift.rigid_point_set_registration(x_pt_cld ,y_pt_cld, verbose=true,
            w=0.0, σ²_tol=0.01, q_tol=1.0, max_nb_em_steps=30, print_ending=true)

write_xyz(atoms_y, R * y, cage_y * "_aligned_to_" * cage_x)

	EM step: 1
		objective: 18402.060571107446
		σ² = 1.492326237511174
	EM step: 2
		objective: 10449.044870983
		σ² = 0.7043175096573946
	EM step: 3
		objective: 3256.2527452541653
		σ² = 0.26348590756512374
	EM step: 4
		objective: -4034.2150143792005
		σ² = 0.12463334255387065
	EM step: 5
		objective: -9208.048990139023
		σ² = 0.07152371867097487
	EM step: 6
		objective: -13017.741121099516
		σ² = 0.046911359281435826
	EM step: 7
		objective: -15847.911421277868
		σ² = 0.03425065980486494
	EM step: 8
		objective: -17902.213482401246
		σ² = 0.02736418147779914
	EM step: 9
		objective: -19327.595191113476
		σ² = 0.023528190148661574
	EM step: 10
		objective: -20261.765549843014
		σ² = 0.02138136708969396
	EM step: 11
		objective: -20841.446606378468
		σ² = 0.020188632412958638
	EM step: 12
		objective: -21184.79936885246
		σ² = 0.019534199878927513
	EM step: 13
		objective: -21381.146846777854
		σ² = 0.01918007740318523
	EM step: 14
		objective: -21490.821757717436
		σ² = 0.018993090198

In [28]:
R_, σ², ℓ = read_jld_file_results("B2", "B1")

([0.999909 0.01221 -0.00568276; -0.00427578 0.687938 0.725757; 0.0127709 -0.725667 0.687928], 0.017039375837873377, -22253.44361630675)

In [27]:
R

3×3 Array{Float64,2}:
  0.999858     0.00376881  0.0164155
 -0.0145607    0.683284    0.730007 
 -0.00846522  -0.730143    0.683242 

In [29]:
write_xyz(atoms_y, R_ * y, cage_y * "_aligned_to_" * cage_x)